In [ ]:
df['Название ЖК'] = df['Название ЖК'].fillna(0) #заменяем на ноль, т.к. в описание не везде написан ЖК

**Высота потолка**

In [ ]:
# Высота потолка в метры
def convert_to_metr(height):
    if height > 99:
        return height / 100
    else:
        return height


df['Высота потолков, м'] = df['Высота потолков, м'].apply(convert_to_metr)

In [ ]:
#Заменяем пустые значения на среднюю высоту по станции метро
average_height_by_metro = df.groupby('Метро')['Высота потолков, м'].mean()
def replace_missing_height(row):
    metro = row['Метро']
    height = row['Высота потолков, м']
    if pd.isnull(height):
        return average_height_by_metro.get(metro, height) 
    else:
        return height
df['Высота потолков, м'] = df.apply(replace_missing_height, axis=1)

In [ ]:
#Заменяем выбросы на среднее значение
mean_height = df['Высота потолков, м'].mean()
def replace_extreme_height(height):
    #height = row['Высота потолков, м']
    if height > 5 or height < 2:
        return mean_height  # Возвращаем среднюю высоту по метро или исходное значение, если нет средней высоты
    else:
        return height
df['Высота потолков, м'] = df['Высота потолков, м'].apply(replace_extreme_height)
df['Высота потолков, м'] = df['Высота потолков, м'].fillna(mean_height)
df['Высота потолков, м'] = df['Высота потолков, м'].round(1)

**Лифт**

In [ ]:
df['Этажность'] = df['Дом'].str.extract(r'/(\d+)').astype(int)# Если в доме менее 5 этажей, лифт не обязателен
df['Лифт'] = np.where((df['Этажность'] <= 5) & (df['Лифт'].isna()), 'Нет', df['Лифт'])
# Если в доме от 5 до 9 этажей, то может быть установлен один пассажирский лифт.
df['Лифт'] = np.where((df['Этажность'] > 5) & (df['Этажность'] <= 9) & (df['Лифт'].isna()), 'Пасс(1)', df['Лифт']) # Если в доме от 10 до 20 этажей,  то должно быть установлено не менее двух лифтов, в том числе один из них может быть грузовым.
df['Лифт'] = np.where((df['Этажность'] >= 10) & (df['Этажность'] <= 20) & (df['Лифт'].isna()), 'Пасс (1), Груз (1)', df['Лифт'])
# Если в доме более 20 этажей, дополнительные лифты, включая грузовые, в зависимости от количества этажей и площади здания.
df['Лифт'] = np.where((df['Этажность'] > 20) & (df['Лифт'].isna()), 'Пасс (4)', df['Лифт']) # Если в доме более 20 этажей, обязателен пассажирский лифт, а также возможен грузовой

**Мусоропровод**

In [ ]:
df['Мусоропровод'] = np.where((df['Этажность'] <= 12) & (df['Мусоропровод'].isna()), 'Да', df['Мусоропровод']) # Если в доме до 12 этажей, то ДА

df['Мусоропровод'] = np.where((df['Этажность'] >= 13) & (df['Мусоропровод'].isna()), 'Нет', df['Мусоропровод'])